# Overnight Index Swap Samples

This notebook demonstrates how to access and use the financial-contracts end point of Instrument Pricing Analytics (IPA) in the Refinitiv Data Platform.

##### Learn more

Please visit the [Refinitiv Developer Community Portal](https://developers.refinitiv.com/en/api-catalog/refinitiv-data-platform/refinitiv-data-platform-apis/documentation#ipa-financial-contracts-ir-swap-contracts) to access the API documentation and much more.

You may also visit the [API Playground](https://apidocs.refinitiv.com/Apps/ApiDocs) to get more examples, find  end points and download Swagger files

##### Getting Help and Support

If you have any questions regarding the API usage, please post them on the [Refinitiv Data Platform Q&A Forum](https://community.developers.refinitiv.com/spaces/231/index.html). The Refinitiv Developer Community will be very pleased to help you. 

#### Import the library and connect to the Refinitv Data Platform

This sample works in both desktop and non-desktop environment. The appropriate session can be created based on the constant variable IPA_ENVIRONMENT, that is set by default to DESKTOP for execution in the Refinitiv codebook application. To run this sample in a non-desktop environment, you may set the variable to NON-DESKTOP.

In [2]:
import refinitiv.dataplatform as rdp

# Set IPA_ENVIRONMENT to NON-DESKTOP to use this example in a non desktop environment
# IPA_ENVIRONMENT = "NON-DESKTOP"
IPA_ENVIRONMENT = "DESKTOP"

if (IPA_ENVIRONMENT == "DEKSTOP"):
    session = rdp.DesktopSession(app_key='DEFAULT_CODE_BOOK_APP_KEY')
else:
    %run ".\..\..\credentials.ipynb"
    session = rdp.PlatformSession(APP_KEY, rdp.GrantPassword(RDP_LOGIN, RDP_PASSWORD))

session.open()
endpoint = rdp.Endpoint(session, "https://api.edp.thomsonreuters.com/data/quantitative-analytics/v1/financial-contracts")
print("session opened and end-point set")

session opened and end-point set


### Valuing an Overnight Index Swap

This sample value an existing basis swap defined leg by leg for a complete control of its term and conditions. The 'fields' requested are:
- the market value in cash (CleanMarketValueInDealCcy, MarketValueInDealCcy, AccruedAmountInDealCcy)
- their equivalent in in percent (AccruedPercent,CleanPricePercent,DirtyPricePercent,CleanPricePercent)
- the PV01 (PV01Bp,PV01AmountInDealCcy)
- the names of the zero-coupon curves used (DiscountCurveName,ForwardCurveName)
- the error information (ErrorCode,ErrorMessage)


In [15]:
request_body = {
    
    "fields" : ["InstrumentTag","MarketValueInDealCcy","DirtyPricePercent","FixedRatePercent","SpreadBp","PV01Bp",\
                "PV01AmountInDealCcy","DiscountCurveName","ForwardCurveName","ErrorCode","ErrorMessage"],
    
    "universe" : [
        {
            "instrumentType":"Swap",
            "instrumentDefinition": {
                "instrumentTag":"OIS-EONIA-2Y",
                "startDate":"2020-01-28",
                "tenor":"2Y",
                "legs":[
                {
                    "direction":"Paid",
                    "interestType":"Fixed",
                    "interestPaymentFrequency":"Annual",
                    "interestCalculationMethod":"Dcb_Actual_360",
                    "notionalCcy":"EUR",
                    "fixedRatePercent":-0.35
                },
                {
                    "direction":"Received",
                    "interestType":"Float",
                    "interestPaymentFrequency":"Quarterly",
                    "interestCalculationMethod":"Dcb_Actual_360",
                    "notionalCcy":"EUR",
                    "indexName":"EONIA",
                    "indexTenor":"ON",
                    "SpreadBp":0
                }]
            }
        }],
    
    "pricingParameters": {
        "valuationDate": "2020-10-20T00:00:00Z"
    },
        
    "outputs" : ["Data","Headers"],

}

response = endpoint.send_request(
    method = rdp.Endpoint.RequestMethod.POST,
    body_parameters = request_body)

print(response.data.raw)

{'headers': [{'type': 'String', 'name': 'InstrumentTag'}, {'type': 'Float', 'name': 'MarketValueInDealCcy'}, {'type': 'Float', 'name': 'DirtyPricePercent'}, {'type': 'Float', 'name': 'FixedRatePercent'}, {'type': 'Float', 'name': 'SpreadBp'}, {'type': 'Float', 'name': 'PV01Bp'}, {'type': 'Float', 'name': 'PV01AmountInDealCcy'}, {'type': 'String', 'name': 'DiscountCurveName'}, {'type': 'String', 'name': 'ForwardCurveName'}, {'type': 'String', 'name': 'ErrorCode'}, {'type': 'String', 'name': 'ErrorMessage'}], 'data': [['OIS-EONIA-2Y', -7136.25847942757, -0.713625847942757, -0.35, None, -0.00555244482591661, -0.555244482591661, 'EUR - OIS EONIA', None, '', ''], ['OIS-EONIA-2Y', -8011.86388623891, -0.801186388623891, None, 0.0, -1.28900000500627, -128.900000500627, 'EUR - OIS EONIA', 'EUR - OIS EONIA', '', '']]}


In [16]:
import pandas as pd

headers_name = [h['name'] for h in response.data.raw['headers']]
df = pd.DataFrame(data=response.data.raw['data'], columns=headers_name)
df

,InstrumentTag,MarketValueInDealCcy,DirtyPricePercent,FixedRatePercent,SpreadBp,PV01Bp,PV01AmountInDealCcy,DiscountCurveName,ForwardCurveName,ErrorCode,ErrorMessage
0,OIS-EONIA-2Y,-7136.258479,-0.713626,-0.35,NaN,-0.005552,-0.555244,EUR - OIS EONIA,None,,
1,OIS-EONIA-2Y,-8011.863886,-0.801186,NaN,0.0,-1.289000,-128.900001,EUR - OIS EONIA,EUR - OIS EONIA,,


2021-01-08 18:30:12,019 - Session session.platform - Thread 18400 | MainThread
[Error 400 - invalid_grant] empty error description
